In [1]:
from keras import activations
from keras.applications import VGG16
from keras.models import load_model
from vis.utils import utils

Using TensorFlow backend.


First, load the model that we tuned. We can print out the summary of our model to double-check.

In [2]:
# Load our model
model = load_model('bikes_classification_best.hdf5')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

It's easy to lose track of which layer we're visualizing, but we can find the ID of the layer from its name. We'll be visualizing the softmax output layer.

In [3]:
layer_name = 'output'
layer_idx = utils.find_layer_idx(model, layer_name)
print(layer_idx)

22


In order for these visualization methods to work, we have to convert the softmax activations to linear activations. This code does that, and stores the new model temporarily.

In [4]:
import os
from keras.models import load_model

# Taken from vis package's util file and modified to work with Windows directory structure
def apply_modifications(model):
    """Applies modifications to the model layers to create a new Graph. For example, simply changing
    `model.layers[idx].activation = new activation` does not change the graph. The entire graph needs to be updated
    with modified inbound and outbound tensors because of change in layer building function.

    Args:
        model: The `keras.models.Model` instance.

    Returns:
        The modified model with changes applied. Does not mutate the original `model`.
    """
    # The strategy is to save the modified model and load it back. This is done because setting the activation
    # in a Keras layer doesnt actually change the graph. We have to iterate the entire graph and change the
    # layer inbound and outbound nodes with modified tensors. This is doubly complicated in Keras 2.x since
    # multiple inbound and outbound nodes are allowed with the Graph API.
    model_path = 'C:/Users/Richard/AppData/Local/Temp/temp.h5'
    try:
        model.save(model_path)
        return load_model(model_path)
    finally:
        os.remove(model_path)
        
# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = apply_modifications(model)

# Visualize saliency map and CAM

First, we'll load some of the examples to visualize. These are the examples we use for our final report.

In [5]:
from matplotlib import pyplot as plt
%matplotlib inline
from vis.visualization import visualize_saliency, visualize_cam, overlay
from vis.utils import utils
from keras import activations
import numpy as np

plt.rcParams['figure.figsize'] = (18, 6)

# A demo image from each class
# Original images, IDs: 67, 13855, 14987, 25183, 28869, 41129, 64968, 67781
indices = ["67", "371", "372", "1813", "7716", "13855", "14987", "25183", "28869", "41129", "64968", "67781"]
imgs = []
for c in indices:
    img = utils.load_img('../datasets/visualization/bikesdemo/'+c+".jpg")
    imgs.append(img)

For each image, we'll generate the saliency map for each of the classification output classes, and plot it next to the original input image.

In [6]:
for j in range(len(indices)):
    c = imgs[j]
    for i in range(4):
        plt.figure()
        grads = visualize_saliency(model, layer_idx, filter_indices=i, seed_input=c, backprop_modifier='guided')
        stitched = utils.stitch_images([c,grads], margin=0)
        plt.axis('off')
        plt.imshow(stitched)
        plt.savefig("../datasets/visualization/bikesdemo/saliency/"+indices[j]+"_"+str(i)+".png")
        plt.close()

We repeat the same process for CAM.

In [7]:
import numpy as np
import matplotlib.cm as cm

for j in range(len(indices)):
    c = imgs[j]
    for i in range(4):
        plt.figure()
        grads = visualize_cam(model, layer_idx, filter_indices=i, seed_input=c, backprop_modifier='guided')        
        # Create heatmap and overlay it on the original pic  
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        plt.axis('off')
        plt.imshow(overlay(jet_heatmap[:,:,:,0], c))
        plt.savefig("../datasets/visualization/bikesdemo/cam/"+indices[j]+"_"+str(i)+".png")
        plt.close()